In [2]:
import pandas as pd
import numpy as np

# Lire les données CSV pour chaque année
years = [2015, 2016, 2017]
data = {}
for year in years:
    df = pd.read_csv(f"/Users/claradalon/Documents/GitHub/Sacoche/archive/marathon_results_{year}.csv")  # remplacer par les noms réels des fichiers
    df['Year'] = year  # ajouter la colonne année si ce n'est pas déjà dans le CSV
    data[year] = df

# Vérifions le nombre de lignes pour chaque année
for year, df in data.items():
    print(year, ":", len(df), "coureurs")


2015 : 26598 coureurs
2016 : 26630 coureurs
2017 : 26410 coureurs


In [13]:
import numpy as np

for year, df in data.items():
    # Conversion des temps officiels en secondes (nombre de secondes écoulées)
    def time_to_seconds(t):
        if pd.isna(t) or t == "" or t is None:
            return np.nan  # si pas de temps (DNF par ex.), on garde NaN
        # Supposons le format "H:MM:SS" ou "HH:MM:SS"
        parts = t.split(':')
        # Si le format est H:MM:SS, parts[0] peut être une ou deux chiffres
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
        total_sec = hours*3600 + minutes*60 + seconds
        return total_sec

    df['OfficialSeconds'] = df['Official Time'].apply(time_to_seconds)


In [14]:
df.head()

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division,Year,OfficialSeconds
0,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,...,1:48:19,2:02:53,0:04:57,-,2:09:37,1,1,1,2017,7777
1,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,...,1:48:19,2:03:14,0:04:58,-,2:09:58,2,2,2,2017,7798
2,2,23,"Osako, Suguru",25,M,Machida-City,NaN,JPN,NaN,NaN,...,1:48:31,2:03:38,0:04:59,-,2:10:28,3,3,3,2017,7828
3,3,21,"Biwott, Shadrack",32,M,Mammoth Lakes,CA,USA,NaN,NaN,...,1:48:58,2:04:35,0:05:03,-,2:12:08,4,4,4,2017,7928
4,4,9,"Chebet, Wilson",31,M,Marakwet,NaN,KEN,NaN,NaN,...,1:48:41,2:05:00,0:05:04,-,2:12:35,5,5,5,2017,7955


In [15]:
stats = {}  # dictionnaire pour stocker les stats par année

for year, df in data.items():
    total = len(df)
    # Nombre de finishers (ceux qui ont un temps officiel non manquant)
    finishers = df['OfficialSeconds'].count()
    completion_rate = finishers / total if total > 0 else 0

    # Calcul des statistiques de temps (sur OfficialSeconds non-NaN)
    mean_sec = df['OfficialSeconds'].mean()
    median_sec = df['OfficialSeconds'].median()
    fastest_sec = df['OfficialSeconds'].min()
    slowest_sec = df['OfficialSeconds'].max()

    # Fonction pour formater les secondes en H:MM:SS
    def sec_to_hms(sec):
        if pd.isna(sec):
            return None
        sec = int(sec)
        h = sec // 3600
        m = (sec % 3600) // 60
        s = sec % 60
        return f"{h:d}:{m:02d}:{s:02d}"

    stats[year] = {
        "total_runners": total,
        "completion_rate": completion_rate,
        "mean_time": sec_to_hms(mean_sec),
        "median_time": sec_to_hms(median_sec),
        "fastest_time": sec_to_hms(fastest_sec),
        "slowest_time": sec_to_hms(slowest_sec),
        "segments": []  # on remplira plus tard
    }

# Aperçu des stats globales calculées
from pprint import pprint
pprint(stats)


{2015: {'completion_rate': 1.0,
        'fastest_time': '2:09:17',
        'mean_time': '3:46:25',
        'median_time': '3:39:40',
        'segments': [],
        'slowest_time': '8:06:01',
        'total_runners': 26598},
 2016: {'completion_rate': 1.0,
        'fastest_time': '2:12:45',
        'mean_time': '3:55:02',
        'median_time': '3:48:05',
        'segments': [],
        'slowest_time': '10:30:23',
        'total_runners': 26630},
 2017: {'completion_rate': 1.0,
        'fastest_time': '2:09:37',
        'mean_time': '3:58:03',
        'median_time': '3:51:39',
        'segments': [],
        'slowest_time': '7:58:14',
        'total_runners': 26410}}


In [16]:
def categorize_time(sec):
    """Retourne la catégorie de niveau en fonction du temps (en sec)."""
    if pd.isna(sec):
        return None  # pas de temps => pas de catégorie (DNF ou non partant)
    if sec < 3*3600:
        return "Elite"
    elif sec < 4*3600:
        return "Avancé"
    elif sec < 5*3600:
        return "Intermédiaire"
    else:
        return "Débutant"

for year, df in data.items():
    df['Level'] = df['OfficialSeconds'].apply(categorize_time)


In [19]:
# Définition des tranches d'âge
bins = [18, 30, 40, 50, 60, 70, np.inf]  # np.inf pour couvrir tout âge >= 70
labels = ["18-29", "30-39", "40-49", "50-59", "60-69", "70+"]

for year, df in data.items():
    # Créer la colonne 'AgeRange' en coupant l'âge selon nos intervalles
    df['AgeRange'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)


In [21]:
for year, df in data.items():
    # Filtrer les finishers uniquement (OfficialSeconds non NaN)
    finished = df.dropna(subset=['OfficialSeconds'])
    # Regrouper par AgeRange, Gender, Level
    group = finished.groupby(['AgeRange', 'Gender', 'Level'])
    # Calculer le count, mean, median, min, max des temps par groupe
    agg_stats = group['OfficialSeconds'].agg(['count', 'mean', 'median', 'min', 'max']).reset_index()
    # Renommer les colonnes pour plus de clarté
    agg_stats.rename(columns={
        'count': 'count',
        'mean': 'mean_sec',
        'median': 'median_sec',
        'min': 'min_sec',
        'max': 'max_sec'
    }, inplace=True)
    
    # Pour chaque ligne de agg_stats, formater les temps en H:MM:SS et ajouter au JSON
    for _, row in agg_stats.iterrows():
        segment_info = {
            "age_range": str(row['AgeRange']),  # convertir de catégorie à str
            "gender": row['Gender'],
            "level": row['Level'],
            "count": int(row['count']),
            "mean_time": sec_to_hms(row['mean_sec']),
            "median_time": sec_to_hms(row['median_sec']),
            "fastest_time": sec_to_hms(row['min_sec']),
            "slowest_time": sec_to_hms(row['max_sec'])
        }
        stats[year]["segments"].append(segment_info)


In [22]:
df.head()

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,Pace,Proj Time,Official Time,Overall,Gender,Division,Year,OfficialSeconds,Level,AgeRange
0,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,...,0:04:57,-,2:09:37,1,1,1,2017,7777,Elite,18-29
1,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,...,0:04:58,-,2:09:58,2,2,2,2017,7798,Elite,30-39
2,2,23,"Osako, Suguru",25,M,Machida-City,NaN,JPN,NaN,NaN,...,0:04:59,-,2:10:28,3,3,3,2017,7828,Elite,18-29
3,3,21,"Biwott, Shadrack",32,M,Mammoth Lakes,CA,USA,NaN,NaN,...,0:05:03,-,2:12:08,4,4,4,2017,7928,Elite,30-39
4,4,9,"Chebet, Wilson",31,M,Marakwet,NaN,KEN,NaN,NaN,...,0:05:04,-,2:12:35,5,5,5,2017,7955,Elite,30-39


In [23]:
import json

# Exporter le dictionnaire stats vers un fichier JSON
with open("stats_marathon_2015_2017.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=4)


In [7]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import logging
from typing import Dict, List, Any, Optional

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class MarathonDataProcessor:
    """
    Processeur de données pour les résultats du Marathon de Boston
    """
    
    def __init__(self):
        self.data: Dict[int, pd.DataFrame] = {}
        self.stats: Dict[str, Any] = {}
        
        # Configuration des catégories
        self.age_bins = [18, 30, 40, 50, 60, 70, np.inf]
        self.age_labels = ["18-29", "30-39", "40-49", "50-59", "60-69", "70+"]
        self.levels = ["Elite", "Avancé", "Intermédiaire", "Débutant"]
        
    def load_data(self, data_directory: str = "") -> None:
        """
        Charge les données CSV pour chaque année
        """
        years = [2015, 2016, 2017]
        data_dir = Path(data_directory)
        
        if not data_dir.exists():
            raise FileNotFoundError(f"Répertoire de données '{data_directory}' introuvable")
        
        for year in years:
            file_path = data_dir / f"/Users/claradalon/Documents/GitHub/Sacoche/archive/marathon_results_{year}.csv"  
            
            if not file_path.exists():
                logger.warning(f"Fichier {file_path} introuvable, ignoré")
                continue
                
            try:
                logger.info(f"Chargement des données {year}...")
                df = pd.read_csv(file_path)
                df['Year'] = year
                
                # Validation basique
                required_columns = ['Official Time', 'Age', 'M/F']
                missing_cols = [col for col in required_columns if col not in df.columns]
                if missing_cols:
                    logger.error(f"Colonnes manquantes dans {year}: {missing_cols}")
                    continue
                
                self.data[year] = df
                logger.info(f"✅ {year}: {len(df)} coureurs chargés")
                
            except Exception as e:
                logger.error(f"Erreur lors du chargement de {year}: {e}")
    
    @staticmethod
    def time_to_seconds(time_str: Any) -> Optional[float]:
        """
        Convertit un temps au format H:MM:SS en secondes
        """
        if pd.isna(time_str) or time_str == "" or time_str is None:
            return np.nan
        
        try:
            if isinstance(time_str, str):
                parts = time_str.split(':')
                if len(parts) != 3:
                    return np.nan
                
                hours = int(parts[0])
                minutes = int(parts[1])
                seconds = int(parts[2])
                
                # Validation des valeurs
                if minutes >= 60 or seconds >= 60:
                    return np.nan
                
                return hours * 3600 + minutes * 60 + seconds
            return np.nan
        except (ValueError, AttributeError):
            return np.nan
    
    @staticmethod
    def sec_to_hms(seconds: float) -> Optional[str]:
        """
        Convertit les secondes en format H:MM:SS
        """
        if pd.isna(seconds):
            return None
        
        seconds = int(seconds)
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        secs = seconds % 60
        return f"{hours:d}:{minutes:02d}:{secs:02d}"
    
    def categorize_performance_level(self, seconds: float) -> Optional[str]:
        """
        Détermine le niveau de performance basé sur le temps
        """
        if pd.isna(seconds):
            return None
        
        if seconds < 3 * 3600:  # < 3h
            return "Elite"
        elif seconds < 4 * 3600:  # 3h-4h
            return "Avancé"
        elif seconds < 5 * 3600:  # 4h-5h
            return "Intermédiaire"
        else:  # > 5h
            return "Débutant"
    
    def process_data(self) -> None:
        """
        Traite les données brutes et ajoute les colonnes calculées
        """
        if not self.data:
            raise ValueError("Aucune donnée chargée. Utilisez load_data() d'abord.")
        
        for year, df in self.data.items():
            logger.info(f"Traitement des données {year}...")
            
            # Conversion des temps en secondes
            df['OfficialSeconds'] = df['Official Time'].apply(self.time_to_seconds)
            
            # Catégorisation du niveau de performance
            df['Level'] = df['OfficialSeconds'].apply(self.categorize_performance_level)
            
            # Création des tranches d'âge
            df['AgeRange'] = pd.cut(
                df['Age'], 
                bins=self.age_bins, 
                labels=self.age_labels, 
                right=False
            )
            
            # Standardisation de la colonne Genre
            df['Gender'] = df['M/F'].map({'M': 1, 'F': 2}).fillna(0)
            
            # Nettoyage des données aberrantes
            valid_times = df['OfficialSeconds'].between(
                60 * 60,  # 1 heure minimum
                12 * 60 * 60  # 12 heures maximum
            )
            invalid_count = (~valid_times & df['OfficialSeconds'].notna()).sum()
            if invalid_count > 0:
                logger.warning(f"{year}: {invalid_count} temps aberrants supprimés")
                df.loc[~valid_times, 'OfficialSeconds'] = np.nan
                df.loc[~valid_times, 'Level'] = None
            
            logger.info(f"✅ {year}: traitement terminé")
    
    def calculate_statistics(self) -> None:
        """
        Calcule les statistiques globales et segmentées pour chaque année
        """
        for year, df in self.data.items():
            logger.info(f"Calcul des statistiques {year}...")
            
            # Statistiques globales
            total_runners = len(df)
            finishers = df['OfficialSeconds'].count()
            completion_rate = finishers / total_runners if total_runners > 0 else 0
            
            # Statistiques de temps (uniquement sur les finishers)
            finished_times = df['OfficialSeconds'].dropna()
            
            if len(finished_times) == 0:
                logger.warning(f"{year}: Aucun temps valide trouvé")
                continue
            
            year_stats = {
                "total_runners": int(total_runners),
                "completion_rate": float(completion_rate),
                "mean_time": self.sec_to_hms(finished_times.mean()),
                "median_time": self.sec_to_hms(finished_times.median()),
                "fastest_time": self.sec_to_hms(finished_times.min()),
                "slowest_time": self.sec_to_hms(finished_times.max()),
                "segments": []
            }
            
            # Statistiques segmentées
            segments = self._calculate_segments(df)
            year_stats["segments"] = segments
            
            self.stats[str(year)] = year_stats
            
            logger.info(f"✅ {year}: {len(segments)} segments calculés")
    
    def _calculate_segments(self, df: pd.DataFrame) -> List[Dict[str, Any]]:
        """
        Calcule les statistiques par segment (âge, genre, niveau)
        """
        segments = []
        
        # Filtrer uniquement les finishers avec des données complètes
        finished = df.dropna(subset=['OfficialSeconds', 'AgeRange', 'Gender', 'Level'])
        
        if len(finished) == 0:
            return segments
        
        # Groupement par AgeRange, Gender, Level
        grouped = finished.groupby(['AgeRange', 'Gender', 'Level'])
        
        for (age_range, gender, level), group in grouped:
            if len(group) == 0:
                continue
            
            times = group['OfficialSeconds']
            
            segment = {
                "age_range": str(age_range),
                "gender": int(gender),
                "level": level,
                "count": len(group),
                "mean_time": self.sec_to_hms(times.mean()),
                "median_time": self.sec_to_hms(times.median()),
                "fastest_time": self.sec_to_hms(times.min()),
                "slowest_time": self.sec_to_hms(times.max())
            }
            
            segments.append(segment)
        
        return segments
    
    def export_json(self, output_file: str = "stats_marathon_2015_2017.json") -> None:
        """
        Exporte les statistiques vers un fichier JSON
        """
        if not self.stats:
            raise ValueError("Aucune statistique calculée. Utilisez calculate_statistics() d'abord.")
        
        try:
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.stats, f, ensure_ascii=False, indent=4)
            
            logger.info(f"✅ Statistiques exportées vers {output_file}")
            
            # Résumé de l'export
            total_segments = sum(len(year_data["segments"]) for year_data in self.stats.values())
            total_runners = sum(year_data["total_runners"] for year_data in self.stats.values())
            
            logger.info(f"📊 Résumé: {len(self.stats)} années, {total_segments} segments, {total_runners:,} coureurs total")
            
        except Exception as e:
            logger.error(f"Erreur lors de l'export: {e}")
            raise
    
    def generate_summary_report(self) -> str:
        """
        Génère un rapport de synthèse des données
        """
        if not self.stats:
            return "Aucune donnée à reporter"
        
        report = ["=== RAPPORT DE SYNTHÈSE MARATHON DE BOSTON ===\n"]
        
        for year, data in self.stats.items():
            report.append(f"📅 ANNÉE {year}")
            report.append(f"   👥 Participants: {data['total_runners']:,}")
            report.append(f"   🏁 Taux de finition: {data['completion_rate']*100:.1f}%")
            report.append(f"   ⏱️  Temps moyen: {data['mean_time']}")
            report.append(f"   🥇 Plus rapide: {data['fastest_time']}")
            report.append(f"   📊 Segments: {len(data['segments'])}")
            
            # Top niveaux
            level_counts = {}
            for segment in data['segments']:
                level = segment['level']
                level_counts[level] = level_counts.get(level, 0) + segment['count']
            
            if level_counts:
                top_level = max(level_counts, key=level_counts.get)
                report.append(f"   🏆 Niveau dominant: {top_level} ({level_counts[top_level]:,} coureurs)")
            
            report.append("")
        
        return "\n".join(report)

def main():
    """
    Fonction principale pour traiter les données du marathon
    """
    try:
        # Initialisation du processeur
        processor = MarathonDataProcessor()
        
        # Traitement des données
        
        processor.load_data("")  # Changez le chemin si nécessaire
        processor.process_data()
        processor.calculate_statistics()
        
        # Export JSON
        processor.export_json()
        
        # Affichage du rapport
        print(processor.generate_summary_report())
        
        logger.info("🎉 Traitement terminé avec succès!")
        
    except Exception as e:
        logger.error(f"❌ Erreur durante le traitement: {e}")
        raise

if __name__ == "__main__":
    main()

2025-05-28 17:31:17,591 - INFO - Chargement des données 2015...
2025-05-28 17:31:17,673 - INFO - ✅ 2015: 26598 coureurs chargés
2025-05-28 17:31:17,673 - INFO - Chargement des données 2016...
2025-05-28 17:31:17,750 - INFO - ✅ 2016: 26630 coureurs chargés
2025-05-28 17:31:17,751 - INFO - Chargement des données 2017...
2025-05-28 17:31:17,830 - INFO - ✅ 2017: 26410 coureurs chargés
2025-05-28 17:31:17,830 - INFO - Traitement des données 2015...
2025-05-28 17:31:17,882 - INFO - ✅ 2015: traitement terminé
2025-05-28 17:31:17,882 - INFO - Traitement des données 2016...
2025-05-28 17:31:17,922 - INFO - ✅ 2016: traitement terminé
2025-05-28 17:31:17,922 - INFO - Traitement des données 2017...
2025-05-28 17:31:17,962 - INFO - ✅ 2017: traitement terminé
2025-05-28 17:31:17,962 - INFO - Calcul des statistiques 2015...
2025-05-28 17:31:17,992 - INFO - ✅ 2015: 43 segments calculés
2025-05-28 17:31:17,992 - INFO - Calcul des statistiques 2016...
2025-05-28 17:31:18,016 - INFO - ✅ 2016: 44 segments

=== RAPPORT DE SYNTHÈSE MARATHON DE BOSTON ===

📅 ANNÉE 2015
   👥 Participants: 26,598
   🏁 Taux de finition: 100.0%
   ⏱️  Temps moyen: 3:46:25
   🥇 Plus rapide: 2:09:17
   📊 Segments: 43
   🏆 Niveau dominant: Avancé (16,253 coureurs)

📅 ANNÉE 2016
   👥 Participants: 26,630
   🏁 Taux de finition: 100.0%
   ⏱️  Temps moyen: 3:55:02
   🥇 Plus rapide: 2:12:45
   📊 Segments: 44
   🏆 Niveau dominant: Avancé (15,355 coureurs)

📅 ANNÉE 2017
   👥 Participants: 26,410
   🏁 Taux de finition: 100.0%
   ⏱️  Temps moyen: 3:58:03
   🥇 Plus rapide: 2:09:37
   📊 Segments: 44
   🏆 Niveau dominant: Avancé (14,173 coureurs)

